FIRST EXPLORATION OF FASTF1

In [1]:
#Imports
import fastf1
import pandas as pd
import matplotlib.pyplot as plt

#Settings
fastf1.Cache.enable_cache('cache')
pd.set_option('display.max_columns', None) 

print("Setup Complete")


Setup Complete


In [2]:
#load data
session = fastf1.get_session(2025, 'Monaco', 'R')
session.load()

#Getting data
laps = session.laps
results = session.results
drivers = session.drivers
schedule = fastf1.get_event_schedule(2025)

print(results[['DriverNumber', 'Abbreviation', 'TeamName','ClassifiedPosition', 'GridPosition']])
print(schedule[['EventDate', 'EventName', 'Location']])

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '16', '81', '1', '44', '6', '31', '30', '23', '55', '63', '87', '43', '5', '18', '27', '22', '12', '14', '10']


   DriverNumber Abbreviation         TeamName ClassifiedPosition  GridPosition
4             4          NOR          McLaren                  1           1.0
16           16          LEC          Ferrari                  2           2.0
81           81          PIA          McLaren                  3           3.0
1             1          VER  Red Bull Racing                  4           4.0
44           44          HAM          Ferrari                  5           7.0
6             6          HAD     Racing Bulls                  6           5.0
31           31          OCO     Haas F1 Team                  7           8.0
30           30          LAW     Racing Bulls                  8           9.0
23           23          ALB         Williams                  9          10.0
55           55          SAI         Williams                 10          11.0
63           63          RUS         Mercedes                 11          14.0
87           87          BEA     Haas F1 Team       

In [ ]:
#filtering data
drivers_laps = laps.pick_drivers('NOR')
fastest_lap = laps.pick_fastest()
qualy_laps = laps.pick_quicklaps()

print(drivers_laps[['Driver', 'LapTime']].head(10))
print(fastest_lap)


In [ ]:
#calculations/filtering
print(laps.info())
avg_times = laps.groupby('Team')['LapTime'].mean()
print(avg_times.sort_values())

In [ ]:
from fastf1.core import Laps
import fastf1.plotting
from timple.timedelta import strftimedelta

#plotting
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme=None)
drivers = pd.unique(session.laps['Driver'])

list_fastest_laps = list()
for drv in drivers:
    drvs_fastest_lap = session.laps.pick_drivers(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)

fastest_laps = Laps(list_fastest_laps) \
    .sort_values(by='LapTime') \
    .reset_index(drop=True)

pole_lap = fastest_laps.pick_fastest()
fastest_laps['LapTimeDelta'] = fastest_laps['LapTime'] - pole_lap['LapTime']

print(fastest_laps.columns)
print(fastest_laps[['Driver', 'LapTime', 'LapTimeDelta']])

team_colors = list()
for index, lap in fastest_laps.iterlaps():
    color = fastf1.plotting.get_team_color(lap['Team'], session=session)
    team_colors.append(color)

fig, ax = plt.subplots()
ax.barh(fastest_laps.index, fastest_laps['LapTimeDelta'],
        color=team_colors, edgecolor='grey')
ax.set_yticks(fastest_laps.index)
ax.set_yticklabels(fastest_laps['Driver'])

# show fastest at the top
ax.invert_yaxis()

# draw vertical lines behind the bars
ax.set_axisbelow(True)
ax.xaxis.grid(True, which='major', linestyle='--', color='black', zorder=-1000)

lap_time_string = strftimedelta(pole_lap['LapTime'], '%m:%s.%ms')

plt.suptitle(f"{session.event['EventName']} {session.event.year} Qualifying\n"
             f"Fastest Lap: {lap_time_string} ({pole_lap['Driver']})")

plt.show()

SECOND EXPLORATION OF FASTF1

In [ ]:
races_to_test = ['Bahrain', 'Australia', 'Canada']

for race in races_to_test:
    try:
        print(f"\n{'='*50}")
        print(f"Loading {race}")

        session = fastf1.get_session(2025 , race , 'R')
        session.load()

        print(f"{race}: {len(session.laps)} laps,  {len(session.drivers)} drivers ")

    except Exception as e:
        print(f"Error loading {race}: {e}")

In [ ]:
# Database Planning

"""
DATABASE SCHEMA NOTES:

Based on exploration

RACES table:
- year (from session.event.year)
- race_name (from session.event.EventName)
- event_date (from session.event.EventDate)
- location (from session.event.Location)
- country (from session.event.Country)

DRIVERS table:
- driver_code (from 'Driver' column - e.g., 'VER')
- driver_number (from 'DriverNumber')
- abbreviation (from results 'Abbreviation')

LAPS table (from session.laps):
- race_id (foreign key)
- driver_id (foreign key)
- lap_number (from 'LapNumber')
- lap_time_seconds (from 'LapTime' converted)
- compound (from 'Compound')
- tyre_life (from 'TyreLife')
- stint (from 'Stint')
- team (from 'Team')
- is_personal_best (from 'IsPersonalBest')

RACE_RESULTS table (from session.results):
- race_id (foreign key)
- driver_id (foreign key)
- position (from 'ClassifiedPosition')
- grid_position (from 'GridPosition')
- points (from 'Points')
- status (from 'Status')
"""

# Verify these columns exist
print("Checking if planned columns exist...")
print("\nEvent columns I need: " )
needeed_event_cols = ['EventName', 'EventDate', 'Location', 'Country']

for field in needeed_event_cols:
    exists = field in session.event.keys()
    print(f" {field}: {'~' if exists else 'missing'}")

print("\nLaps columns I need:")
needed_lap_cols = ['Driver', 'LapNumber', 'LapTime', 'Compound', 'TyreLife', 'Stint', 'Team', 'IsPersonalBest']
for col in needed_lap_cols:
    exists = col in laps.columns
    print(f"  {col}: {'✓' if exists else '✗ MISSING'}")

print("\nResults columns I need:")
needed_result_cols = ['DriverNumber', 'Abbreviation', 'ClassifiedPosition', 'GridPosition', 'Points', 'Status']
for col in needed_result_cols:
    exists = col in results.columns
    print(f"  {col}: {'✓' if exists else '✗ MISSING'}")

In [ ]:
#data type exploration
print("LapTime type:", laps['LapTime'].dtype )
print("Sample Time", laps['LapTime'].iloc[0])

laps['lapTimeSeconds'] = laps['LapTime'].dt.total_seconds()
print("Converted Time", laps['lapTimeSeconds'].iloc[0])

print("\nNull values in key columns:")
print(laps[['LapTime', 'Compound', 'TyreLife']].isnull().sum())

# See what happens with missing lap times
print("\nRows with missing lap times:")
print(laps[laps['LapTime'].isnull()][['Driver', 'LapNumber', 'TrackStatus']].head())


In [ ]:
# Find pit laps
pit_laps = laps[laps['PitInTime'].notna()]
print(f"Laps with PitOutTime: {laps['PitOutTime'].notna().sum()}")
print(f"Total pit stops: {len(pit_laps)}")


print("\nPit stop data sample:")
print(pit_laps[['Driver', 'LapNumber', 'Compound', 'TyreLife', 'PitInTime', 'PitOutTime']].head())

# Calculate pit stop duration
pit_laps_copy = pit_laps.copy()
pit_laps_copy['PitDuration'] = (pit_laps_copy['PitOutTime'] - pit_laps_copy['PitInTime']).dt.total_seconds()
print("\nAverage pit stop duration by team:")
print(pit_laps_copy.groupby('Team')['PitDuration'].mean().sort_values())


In [8]:
sqlite3 -header -column data/database.db "SELECT * FROM races WHERE year=2025 LIMIT 50;"
sqlite3 -header -column data/database.db "SELECT * FROM drivers LIMIT 50;"
sqlite3 -header -column data/database.db "SELECT * FROM results LIMIT 50;"
sqlite3 -header -column data/database.db "SELECT * FROM laps LIMIT 50;"

SyntaxError: invalid syntax (3935817976.py, line 1)

In [17]:
import sqlite3

# Connect to database (from notebook directory, go up one level)
conn = sqlite3.connect('../data/database.db')
cursor = conn.cursor()

print("Before cleanup:")
cursor.execute("SELECT COUNT(*) FROM races WHERE year=2020")
print(f"Total 2023 races: {cursor.fetchone()[0]}")

# Find duplicates: races with matching dates but different names
cursor.execute("""
    SELECT event_date, COUNT(*), GROUP_CONCAT(race_name, ' | ')
    FROM races 
    WHERE year=2020
    GROUP BY event_date
    HAVING COUNT(*) > 1
""")
duplicates = cursor.fetchall()
print(f"\nDuplicate dates: {len(duplicates)}")
for date, count, names in duplicates:
    print(f"  {date}: {names}")

# Delete races that DON'T contain "Grand Prix" (keep the full names)
print("\nDeleting short-name races and testing events...")
cursor.execute("""
    DELETE FROM races 
    WHERE year=2020
    AND (
        race_name NOT LIKE '%Grand Prix%'
        OR race_name LIKE '%Testing%'
        OR race_name LIKE '%Test%'
    )
""")
deleted = cursor.rowcount
print(f"Deleted {deleted} rows")

# Clean up orphaned results and laps (if any)
cursor.execute("""
    DELETE FROM results 
    WHERE race_id NOT IN (SELECT id FROM races)
""")
orphaned_results = cursor.rowcount

cursor.execute("""
    DELETE FROM laps 
    WHERE race_id NOT IN (SELECT id FROM races)
""")
orphaned_laps = cursor.rowcount

print(f"Cleaned {orphaned_results} orphaned results, {orphaned_laps} orphaned laps")

conn.commit()

print("\nAfter cleanup:")
cursor.execute("SELECT COUNT(*) FROM races WHERE year=2020")
print(f"Total 2023 races: {cursor.fetchone()[0]}")

cursor.execute("SELECT id, race_name, event_date, location FROM races WHERE year=2020 ORDER BY event_date")
races = cursor.fetchall()
print("\nRemaining races:")
for race_id, name, date, location in races:
    print(f"  {race_id:3d} | {name:35s} | {date} | {location}")

conn.close()
print("\n✓ Cleanup complete!")

Before cleanup:
Total 2023 races: 19

Duplicate dates: 1
  2020-07-05 00:00:00.000000: Pre-Season Test 1 | Pre-Season Test 2 | Austrian Grand Prix

Deleting short-name races and testing events...
Deleted 2 rows
Cleaned 40 orphaned results, 2122 orphaned laps

After cleanup:
Total 2023 races: 17

Remaining races:
   11 | Austrian Grand Prix                 | 2020-07-05 00:00:00.000000 | Spielberg
   12 | Styrian Grand Prix                  | 2020-07-12 00:00:00.000000 | Spielberg
   13 | Hungarian Grand Prix                | 2020-07-19 00:00:00.000000 | Budapest
   14 | British Grand Prix                  | 2020-08-02 00:00:00.000000 | Silverstone
   15 | 70th Anniversary Grand Prix         | 2020-08-09 00:00:00.000000 | Silverstone
   16 | Spanish Grand Prix                  | 2020-08-16 00:00:00.000000 | Barcelona
   17 | Belgian Grand Prix                  | 2020-08-30 00:00:00.000000 | Spa-Francorchamps
   18 | Italian Grand Prix                  | 2020-09-06 00:00:00.000000 | Monza

## Clean up 2025 duplicates

The duplicates came from:
1. Running pipeline with short names ("Australia") and full names ("Australian Grand Prix")  
2. Pre-Season Testing being included in the schedule

Solution: Keep only the full "Grand Prix" names and remove testing events.

add the races type to the laps (Qualifying, Race , free practice)